In [2]:
import os 

In [3]:
%pwd

'd:\\DS\\end-to-end-DS-project\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'd:\\DS\\end-to-end-DS-project'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class modelTrainerConfig:
    root_dir : Path
    train_data_dir : Path
    test_data_dir : Path
    model_name : str
    aplha : float
    l1_ratio : float
    target_column : str
    

In [8]:
from src.datascience.constants import *
from src.datascience.utils.common import read_yaml, create_directories

In [14]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> modelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.ElasticNet
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = modelTrainerConfig(
            root_dir = config.root_dir,
            train_data_dir = config.train_data_path,
            test_data_dir = config.test_data_path,
            model_name = config.model_name,
            aplha = params.alpha,
            l1_ratio = params.l1_ratio,
            target_column = schema.name
        )
        
        return model_trainer_config
    
    
            

In [15]:
import pandas as pd
import os
from src.datascience import logger
from sklearn.linear_model import ElasticNet
import joblib
from sklearn.metrics import mean_squared_error, r2_score

In [16]:
class ModelTrainer:
    def __init__(self,config: modelTrainerConfig):
        self.config = config
    
    def train(self):
        train_data = pd.read_csv(self.config.train_data_dir)
        test_data = pd.read_csv(self.config.test_data_dir)
        
        train_x = train_data.drop(columns=[self.config.target_column], axis=1)
        test_x = test_data.drop(columns=[self.config.target_column], axis=1)
        train_y = train_data[self.config.target_column]
        test_y = test_data[self.config.target_column]
        
        lr = ElasticNet(
            alpha=self.config.aplha,
            l1_ratio=self.config.l1_ratio,
            random_state=42
        )
        lr.fit(train_x, train_y)
        
        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))
        

In [17]:
try :
    config = ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(model_trainer_config)
    model_trainer.train()
except Exception as e:
    logger.exception(e)
    raise e

[2025-04-21 13:43:35,380: INFO: common]: yaml file: config\config.yaml loaded successfully
[2025-04-21 13:43:35,382: INFO: common]: yaml file: params.yaml loaded successfully
[2025-04-21 13:43:35,386: INFO: common]: yaml file: schema.yaml loaded successfully
[2025-04-21 13:43:35,388: INFO: common]: created directory at: artifacts
[2025-04-21 13:43:35,390: INFO: common]: created directory at: artifacts/model_trainer
